In [1]:
import numpy as np
import pandas as pd
import pickle 

In [2]:
df= pd.read_csv('dataset/athlete_events.csv')
df_regions= pd.read_csv('dataset/noc_regions.csv')

In [3]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
df["Year"].value_counts()

1992    16413
1988    14676
2000    13821
1996    13780
2016    13688
2008    13602
2004    13443
2012    12920
1972    11959
1984    11588
1976    10502
1968    10479
1964     9480
1952     9358
1960     9235
1980     8937
1948     7480
1936     7401
1956     6434
1924     5693
1928     5574
2014     4891
2010     4402
2006     4382
1920     4292
2002     4109
1912     4040
1998     3605
1932     3321
1994     3160
1908     3101
1900     1936
1906     1733
1904     1301
1896      380
Name: Year, dtype: int64

In [5]:
df_regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [7]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [8]:
df['Medal']=df['Medal'].fillna(0)

In [9]:
df.isnull().sum()

ID            0
Name          0
Sex           0
Age        9474
Height    60171
Weight    62875
Team          0
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal         0
dtype: int64

In [10]:
df=df.merge(df_regions,how='left',on='NOC')

In [11]:
df.drop_duplicates(['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'],inplace=True)

In [12]:
df=df[df['Season']=='Summer'].reset_index(drop=True)

In [13]:
df.shape

(105636, 17)

In [14]:
df['Year']=df['Year'].astype(int)

In [15]:
df.drop(['Season','Games'],axis=1,inplace=True)

In [16]:
df= pd.concat([df, pd.get_dummies(df['Medal'])], axis=1)

In [17]:
medaltally=df.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold',ascending=False).reset_index()

C:\Users\niles\AppData\Local\Temp\ipykernel_9368\486467499.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  medaltally=df.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold',ascending=False).reset_index()


In [18]:
years=list(df['Year'].sort_values().unique())
country=list(df['region'].sort_values().unique())
years.insert(0, 'Overall')
country.insert(0, 'Overall')

In [19]:
medaltally.head()

,region,Gold,Silver,Bronze
0,USA,1035,802,708
1,Russia,592,498,487
2,Germany,444,457,491
3,UK,278,317,300
4,France,234,256,287


In [20]:
newdf=df.drop_duplicates(['Name', 'region'])

In [21]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,City,Sport,Event,Medal,region,notes,0,Bronze,Gold,Silver
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,0,China,NaN,1,0,0,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,0,China,NaN,1,0,0,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,0,Denmark,NaN,1,0,0,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,0,0,1,0
4,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,0,Netherlands,NaN,1,0,0,0


In [22]:
#edition
edition=len(df['Year'].unique())
host=len(df['City'].unique())
sports=len(df['Sport'].unique())
events=len(df['Event'].unique())
athletes=len(df['Name'].unique())
nations=len(df['region'].unique())

In [23]:
sports

52

In [24]:
famous_sport=df['Sport'].value_counts().head(39).to_frame().index

In [25]:
for sport in famous_sport:
    temp=df[df['Sport']==sport]
    temp[temp['Medal']=='Gold']['Age']

In [26]:
df[df['Sex'] == 'F'].groupby('Year').count()['Name'].reset_index()

,Year,Name
0,1900,18
1,1904,9
2,1906,8
3,1908,18
4,1912,39
5,1920,74
6,1924,127
7,1928,171
8,1932,129
9,1936,207


In [27]:
#particiapting nations
df.head()


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,City,Sport,Event,Medal,region,notes,0,Bronze,Gold,Silver
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,0,China,NaN,1,0,0,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,0,China,NaN,1,0,0,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,0,Denmark,NaN,1,0,0,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,0,0,1,0
4,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,0,Netherlands,NaN,1,0,0,0


In [51]:
# nation_medal_tally=df.drop_duplicates(['region','Year'])['Year'].value_counts().reset_index().sort_values('index')
# nation_medal_tally.rename(columns={'index': 'Edition', 'Year': 'Athlete'}, inplace=True)

In [54]:
nation_medal_tally=df[['Name', 'Year','Sport','region']]

In [56]:
nation_medal_tally['total_medals']=df['Bronze']+df['Gold']+df['Silver']

C:\Users\niles\AppData\Local\Temp\ipykernel_9368\3423337502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nation_medal_tally['total_medals']=df['Bronze']+df['Gold']+df['Silver']


In [58]:
nation_medal_tally[nation_medal_tally['region']=='USA'].head()

,Name,Year,Sport,region,total_medals
30343,"Thomas Pelham ""Tom"" Curtis",1896,Athletics,USA,0
30344,"Thomas Pelham ""Tom"" Curtis",1896,Athletics,USA,1
81932,Sumner Paine,1896,Shooting,USA,1
103123,Gardner Boyd Williams,1896,Swimming,USA,0
103122,Gardner Boyd Williams,1896,Swimming,USA,0
